In [2]:
!pip install tensorflow

     -------------------------------------- 276.5/276.5 MB 9.2 MB/s eta 0:00:00
     ------------------------------------- 422.5/422.5 kB 12.9 MB/s eta 0:00:00
     ---------------------------------------- 5.6/5.6 MB 11.5 MB/s eta 0:00:00
     ------------------------------------- 440.8/440.8 kB 26.9 MB/s eta 0:00:00
     ---------------------------------------- 1.7/1.7 MB 8.3 MB/s eta 0:00:00
     ---------------------------------------- 4.2/4.2 MB 7.1 MB/s eta 0:00:00
     ---------------------------------------- 182.1/182.1 kB ? eta 0:00:00
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 2.0
    Uninstalling flatbuffers-2.0:
      Successfully uninstalled flatbuffers-2.0
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.6.1
    Uninstalling tensorboard-data-server-0.6.1:
      Successfully uninstalled tensorboard-data-server-0.6.1
  Attempting uninstall: protobuf
    Found existing installation

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mysql-connector-python 8.0.33 requires protobuf<=3.20.3,>=3.11.0, but you have protobuf 4.23.4 which is incompatible.


In [3]:
import tensorflow

In [4]:
import numpy as np
import pandas as pd
import re
import json
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer


## 데이터 불러오기

In [6]:
data = pd.read_csv("4차년도.csv", encoding='cp949')

In [8]:
data2 = pd.read_csv("5차년도.csv", encoding='cp949')

In [11]:
data.drop(['wav_id', '나이', '성별'], axis=1, inplace=True)

In [15]:
data['상황'].unique()

array(['anger', 'sad', 'fear', 'disgust', 'neutral'], dtype=object)

In [17]:
data['1번 감정'].unique()

array(['Neutral', 'Angry', 'Sadness', 'Disgust', 'Surprise', 'Fear',
       'Happiness'], dtype=object)

In [20]:
emotions = data[['1번 감정', '2번 감정', '3번 감정', '4번 감정', '5번 감정']]

In [23]:
list(emotions.loc[0,:])

['Neutral', 'Angry', 'Neutral', 'Neutral', 'Angry']

## 다수결로 발화문에 대한 감정 결정하기

In [33]:
emotion_data = []

for i in range(len(emotions.index)) :
    emotion = list(emotions.loc[i,:])
    counter = {}
    for value in emotion:
        try: counter[value] += 1
        except: counter[value ] = 1
    emotion_data.append(max(counter, key=counter.get))

In [34]:
emotion_data

['Neutral',
 'Neutral',
 'Angry',
 'Sadness',
 'Sadness',
 'Neutral',
 'Disgust',
 'Neutral',
 'Sadness',
 'Sadness',
 'Sadness',
 'Sadness',
 'Sadness',
 'Sadness',
 'Sadness',
 'Neutral',
 'Sadness',
 'Sadness',
 'Neutral',
 'Neutral',
 'Angry',
 'Neutral',
 'Surprise',
 'Sadness',
 'Sadness',
 'Sadness',
 'Sadness',
 'Sadness',
 'Sadness',
 'Sadness',
 'Sadness',
 'Sadness',
 'Sadness',
 'Sadness',
 'Angry',
 'Sadness',
 'Neutral',
 'Sadness',
 'Neutral',
 'Neutral',
 'Neutral',
 'Sadness',
 'Sadness',
 'Sadness',
 'Sadness',
 'Sadness',
 'Neutral',
 'Sadness',
 'Sadness',
 'Sadness',
 'Sadness',
 'Neutral',
 'Sadness',
 'Sadness',
 'Sadness',
 'Sadness',
 'Angry',
 'Surprise',
 'Neutral',
 'Neutral',
 'Angry',
 'Angry',
 'Angry',
 'Angry',
 'Disgust',
 'Disgust',
 'Sadness',
 'Sadness',
 'Sadness',
 'Sadness',
 'Angry',
 'Angry',
 'Angry',
 'Angry',
 'Angry',
 'Sadness',
 'Sadness',
 'Angry',
 'Neutral',
 'Sadness',
 'Sadness',
 'Sadness',
 'Sadness',
 'Sadness',
 'Sadness',
 'Angr

In [38]:
data = pd.concat([data, pd.DataFrame(emotion_data)], axis=1)

In [39]:
data.drop(['1번 감정', '2번 감정', '3번 감정', '4번 감정', '5번 감정'], axis=1, inplace=True)

In [42]:
data.drop(['1번 감정세기', '2번 감정세기', '3번 감정세기', '4번감정세기','5번 감정세기'],axis=1, inplace=True)

In [44]:
data.columns=['발화문', '상황', '감정']

In [46]:
data['감정'].value_counts()

감정
Sadness      7176
Angry        3899
Neutral      1481
Disgust       980
Fear          862
Happiness     137
Surprise       71
Name: count, dtype: int64

## 전처리 : https://wonhwa.tistory.com/35

In [53]:
data

,발화문,상황,감정
0,"어, 청소 니가 대신 해 줘!",anger,Neutral
1,둘 다 청소 하기 싫어. 귀찮아.,anger,Neutral
2,둘 다 하기 싫어서 화내.,anger,Angry
3,그럼 방세는 어떡해.,anger,Sadness
4,권태긴줄 알았는데 다른 사람이 생겼나보더라고.,sad,Sadness
...,...,...,...
14601,"아, 요즘 룸메랑 너무 자주 싸우게 돼.",anger,Angry
14602,"아, 룸메가 방을 너무 지저분하게 써. 음식물도 막 버리고.",anger,Angry
14603,"뭐 화를 낸 것까진 아니지만, 한 달 전 쯤에 좀 확실하게 얘기를 해뒀거든. 근데 ...",anger,Disgust
14604,"어. 고등학교 동창인데, 같은 동네 오게 돼서 같이 룸메로 살게 됐지.",anger,Neutral


In [61]:
data.tail(20)

,발화문,상황,감정
14586,나 면접 또 떨어졌어.,sad,Sadness
14587,"아, 집 앞 공원이야. 좀 걷고 있어. 머리도 식힐 겸.",sad,Sadness
14588,"어. 여기가 마지막 희망이었거든. 아 다른 친구는 다 붙는데, 나만 떨어진 거 같아...",sad,Sadness
14589,"아, 부끄러워서 어떻게 부모님한테 말씀드려. 그냥 조용히 넘어가려고.",sad,Sadness
14590,아니 밥 생각 없어. 조금 이렇게 걷다가 다음 시즌 준비해야지.,sad,Sadness
14591,"아, 어제 아빠가 또 다치셨어.",anger,Sadness
14592,"아, 친구들이랑 술 드시고 계단 올라오시다가 허리를 삐끗하신 모양이야.",anger,Sadness
14593,일단은. 일단은 입원해서 치료 받으셔야 한대. 끝나면 통원 치료도 하셔야되고.,anger,Sadness
14594,한 일주일에 다섯번은 드시는 거 같애. 친구들이 워낙 술을 좋아하셔서.,anger,Sadness
14595,그 문제 때문에 부모님이 많이 다투시지. 아 근데 이제 포기야 포기. 거의 뭐 말을...,anger,Neutral


In [50]:
#전처리 함수 만들기
def preprocessing(review, okt, remove_stopwords = False, stop_words =[]):
    #함수인자설명
    # review: 전처리할 텍스트
    # okt: okt객체를 반복적으로 생성하지 않고 미리 생성 후 인자로 받음
    # remove_stopword: 불용어를 제거할지 여부 선택. 기본값 False
    # stop_words: 불용어 사전은 사용자가 직접 입력, 기본값 빈 리스트

    # 1. 한글 및 공백 제외한 문자 모두 제거
    review_text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]','',review)
    
    #2. okt 객체를 활용하여 형태소 단어로 나눔
    word_review = okt.morphs(review_text,stem=True)
    
    if remove_stopwords:
    #3. 불용어 제거(선택)
        word_review = [token for token in word_review if not token in stop_words]
        return word_review

In [55]:
# 전체 텍스트 전처리
stop_words = ['은','는','이','가','하','아','것','들','의','있','되','수','보','주','등','한']
okt = Okt()
clean_data = []

for review in data['발화문']:
  # 리뷰가 문자열인 경우만 전처리 진행
    if type(review) == str:
        clean_data.append(preprocessing(review,okt,remove_stopwords=True,stop_words= stop_words))
    else:
        clean_data.append([]) #str이 아닌 행은 빈칸으로 놔두기

clean_data[:4]


[['어', '청소', '니', '대신', '해', '주다'],
 ['둘', '다', '청소', '하다', '싫다', '귀찮다'],
 ['둘', '다', '하다', '싫다', '화내'],
 ['그렇다', '방', '세다', '어떡하다']]

## 벡터화

In [56]:
# 인덱스 벡터 변환 후 일정 길이 넘어가거나 모자라는 문장 패딩처리
tokenizer = Tokenizer()
tokenizer.fit_on_texts(clean_data)
sequences = tokenizer.texts_to_sequences(clean_data)

word_vocab = tokenizer.word_index #단어사전형태
MAX_SEQUENCE_LENGTH = 8 #문장 최대 길이

#학습 데이터
inputs = pad_sequences(sequences,maxlen=MAX_SEQUENCE_LENGTH,padding='post')

#학습 데이터 라벨 벡터화
labels = np.array(data['감정'])

In [57]:
inputs

array([[  40,   73,  466, ...,  187,    0,    0],
       [ 659,   47,   73, ...,  790,    0,    0],
       [ 659,   47,    1, ...,    0,    0,    0],
       ...,
       [  19,  146,  407, ...,  523,   99,   30],
       [  24,   15,  122, ...,  177,   95,   15],
       [ 283, 1557,  335, ...,    0,    0,    0]])

In [58]:
labels

array(['Neutral', 'Neutral', 'Angry', ..., 'Disgust', 'Neutral',
       'Neutral'], dtype=object)